In [8]:
import requests
import json
import pandas as pd

In [9]:
res = requests.get("https://cdn-api.co-vin.in/api/v2/admin/location/states")

In [10]:
states = pd.DataFrame(json.loads(res.text)["states"])

In [11]:
states.to_csv("../data/states.csv", index = False) #sates with id 

In [12]:
df_states = pd.read_csv("../data/states.csv")

In [13]:
df_states.head(5)

,state_id,state_name
0,1,Andaman and Nicobar Islands
1,2,Andhra Pradesh
2,3,Arunachal Pradesh
3,4,Assam
4,5,Bihar


In [20]:
df_district_all = None
for state_id in df_states["state_id"].values.tolist():
    state_name = df_states.loc[df_states['state_id'] == state_id, "state_name"].item()
    res = requests.get("https://cdn-api.co-vin.in/api/v2/admin/location/districts/{}".format(state_id))
    df_district = pd.DataFrame(json.loads(res.text)["districts"])
    df_district["state_name"] = state_name
    if df_district_all is None:
        df_district_all = df_district
    else:
        df_district_all = pd.concat([df_district_all, df_district], ignore_index=True)

df_district_all = df_district_all.sort_values("district_name")
df_district_all.to_csv("../data/districts.csv", index=False)    #District_id

In [22]:
df_district_all

,district_id,district_name,state_name
598,582,Adilabad,Telangana
326,320,Agar,Madhya Pradesh
324,796,Agatti Island,Lakshadweep
639,622,Agra,Uttar Pradesh
158,154,Ahmedabad,Gujarat
...,...,...,...
307,285,Yadgir,Karnataka
220,197,Yamunanagar,Haryana
492,478,Yanam,Puducherry
411,368,Yavatmal,Maharashtra


In [30]:
df_district = df_district_all.loc[df_district_all["state_name"]== "Gujarat"]
df_district

,district_id,district_name,state_name
4,154,Ahmedabad,Gujarat
5,770,Ahmedabad Corporation,Gujarat
21,174,Amreli,Gujarat
24,179,Anand,Gujarat
32,158,Aravalli,Gujarat
57,159,Banaskantha,Gujarat
90,180,Bharuch,Gujarat
91,175,Bhavnagar,Gujarat
92,771,Bhavnagar Corporation,Gujarat
110,176,Botad,Gujarat


In [25]:
df_district_all = pd.read_csv("../data/districts.csv")
selected_state = "Gujarat"
district_list = df_district_all["district_name"].tolist()
district_list

['Adilabad',
 'Agar',
 'Agatti Island',
 'Agra',
 'Ahmedabad',
 'Ahmedabad Corporation',
 'Ahmednagar',
 'Aizawl East',
 'Aizawl West',
 'Ajmer',
 'Akola',
 'Alappuzha',
 'Aligarh',
 'Alipurduar District',
 'Alirajpur',
 'Almora',
 'Alwar',
 'Ambala',
 'Ambedkar Nagar',
 'Amethi',
 'Amravati',
 'Amreli',
 'Amritsar',
 'Amroha',
 'Anand',
 'Anantapur',
 'Anantnag',
 'Angul',
 'Anjaw',
 'Anuppur',
 'Aranthangi',
 'Araria',
 'Aravalli',
 'Ariyalur',
 'Arwal',
 'Ashoknagar',
 'Attur',
 'Auraiya',
 'Aurangabad',
 'Aurangabad ',
 'Ayodhya',
 'Azamgarh',
 'BBMP',
 'Badaun',
 'Bagalkot',
 'Bageshwar',
 'Baghpat',
 'Bahraich',
 'Baksa',
 'Balaghat',
 'Balangir',
 'Balarampur',
 'Balasore',
 'Ballia',
 'Balod',
 'Baloda bazar',
 'Balrampur',
 'Banaskantha',
 'Banda',
 'Bandipore',
 'Bangalore Rural',
 'Bangalore Urban',
 'Banka',
 'Bankura',
 'Banswara',
 'Barabanki',
 'Baramulla',
 'Baran',
 'Bareilly',
 'Bargarh',
 'Barmer',
 'Barnala',
 'Barpeta',
 'Barwani',
 'Basirhat HD (North 24 Parganas)

## Search by district id 

In [13]:
#Example

district_id = "240" #Ranchi 
date_from = "01-05-2021"

In [14]:
URL = "https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByDistrict?district_id={}&date={}" \
      .format(district_id,date_from)
res = requests.get(URL)

In [15]:
calender_df = pd.DataFrame(json.loads(res.text)["centers"])

In [16]:
calender_df = calender_df[['name', 'state_name', 'district_name',
       'pincode', 'fee_type','vaccine_fees']]

In [17]:
calender_df.columns

Index(['name', 'state_name', 'district_name', 'pincode', 'fee_type',
       'vaccine_fees'],
      dtype='object')

In [18]:
calender_df.to_csv("../data/vaccine_deatils_7days.csv", index=False)

In [19]:
calender_df.head(10)

,name,state_name,district_name,pincode,fee_type,vaccine_fees
0,Resaldar Baba Doranda,Jharkhand,Ranchi,834001,Free,NaN
1,LAPUNG COVAXIN,Jharkhand,Ranchi,835234,Free,NaN
2,SEVA SADAN,Jharkhand,Ranchi,834001,Paid,"[{'vaccine': 'COVISHIELD', 'fee': '0'}]"
3,TAMAR PANCHAYAT,Jharkhand,Ranchi,835225,Free,NaN
4,JAGUAR,Jharkhand,Ranchi,835222,Free,NaN
5,GURUDWARA PISKA MORE,Jharkhand,Ranchi,834005,Free,NaN
6,NAMKUM PANCHYAT,Jharkhand,Ranchi,834010,Free,NaN
7,JAGARNATH HOSPITAL,Jharkhand,Ranchi,834001,Paid,"[{'vaccine': 'COVAXIN', 'fee': '250'}]"
8,DSP HQ,Jharkhand,Ranchi,834001,Free,NaN
9,BOOTI GOVT SCHOOL,Jharkhand,Ranchi,834001,Free,NaN


In [35]:
import os

In [36]:
os.path.dirname(os.path.abspath(__file__))


NameError: name '__file__' is not defined